**Install Azure Cognitive Search SDK**

In [1]:
!pip install --index-url=https://pkgs.dev.azure.com/azure-sdk/public/_packaging/azure-sdk-for-python/pypi/simple/ azure-search-documents==11.4.0a20230509004
!pip install azure-identity

Looking in indexes: https://pkgs.dev.azure.com/azure-sdk/public/_packaging/azure-sdk-for-python/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 7.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 11.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 7.8 MB/s eta 0:00:00


**Import required libraries**

In [1]:
import os, json
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

**Configure OpenAI and vector store settings**

In [2]:
openai.api_type = os.getenv('OPENAI_API_TYPE')
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_version = os.getenv('OPENAI_API_VERSION')
# #openai.api_key = os.getenv('OPENAI_API_KEY')
model: str = "text-embedding-ada-002"
vector_store_address = os.getenv('AZURE_SEARCH_ENDPOINT')
vector_store_password = os.getenv('AZURE_SEARCH_ADMIN_KEY')
index_name: str = "langchain-vector-demo"

**Create embeddings and vector store instances**

In [16]:
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(model=model, chunk_size=1)
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [5]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

In [6]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("data/SparkOfGPT-4.pdf")
# pages = loader.load_and_split()

**Insert text and embeddings into vector store**

In [17]:
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Get a vector representation of a given input that can be easily consumed by machine learning models and algorithms. Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


['MDkzN2Q5ODYtMzA2NS00ZmNiLTljOTAtNWViNzhhYzAzYmNh',
 'NjYyY2M4NmQtMmM4Mi00MTk1LWI5ODEtN2E2ZjQ2MzA1ODcx',
 'ZWIxNzk5Y2UtODgwYy00OWI2LWFjODEtODQwMTIyNDc1MTdh',
 'NmVmODhiYzYtNDliZS00ZjlmLTk1ODUtZWRlNGM4OWZkMGZi',
 'OGM4MjU5OGUtYmIyZi00NTNjLWE4NjMtZjE3NWY4YTljZjY4',
 'NGI3MmM0ZDEtZDNjNi00OThmLTliNWQtYWJjMTNiMTUxZjEx',
 'MDM5MTk1NmQtOGNhZi00ODk2LTk3YjYtMGIzM2Y2MmZmZTNj',
 'YTk2OGY3ZWQtYzY2NC00MWZmLWE2YWMtZGZjZjAwMmJkZjQ2',
 'OGU5ZDcxMzUtMjU1MC00ZDNjLWI2YjQtZWNjY2QxMTdhNTM2',
 'ZjEzMmQ0NWUtNzc1ZS00MjA4LWE5ZDctZGNkNWYyY2QwY2Ri',
 'MGQ4MzNiYWQtMDM4Ni00M2QxLWFjYjMtNGUyZjI5YzcyOTNk',
 'M2Q5NWFiYWEtYzM5My00OTk5LWJmMzItYWU1MDQzMjQwOTQy',
 'MTJiOTk0YTctNTNiOS00NzUwLTg5YWItM2NhYmY3NDgzNDkz',
 'NjU3OWY4ODAtMTA1Zi00MTBiLWI0ZjAtMWY4NTY5YmI4ZjNk',
 'MjIyOGViZDQtOTYyOC00YzNhLWJmZTMtMTA5YjQ1NGUzMTk3',
 'OTAzYzg3NjktNzRjYi00NmVhLWIyYmQtZmVmMmJmYzA0YzVm',
 'YmNmNjg0MDUtM2U0Ny00MmMzLWI1OGQtYjAyNGExN2UzNGJh',
 'ZDVlNzkxMjMtYjI2ZS00Y2NkLTkwMDItZmI0MTI3NzkxNzk5',
 'NDc3YTUwMWYtZjA3Yi00YWZmLWI5OGMtNWE2ZmE1NDY0

**Perform a vector similarity search**

In [12]:
# Perform a similarity search
docs = vector_store.similarity_search(
    query="What are the areas of improvement for GPT-4?",
    k=3,
    search_type="similarity",
)
print(docs[0].page_content)

Figure 1.5: GPT-4 passes mock technical interviews on LeetCode. GPT-4 could potentially be hired
as a software engineer3.
preliminary tests (see [Ope23] for much more) on the multiple choice component (majority of the score)
of the US Medical Licensing Exam Step 1, 2, and 3 with an accuracy around 80% in each. A similar
preliminary test of GPT-4's competency on the Multistate Bar Exam showed an accuracy above 70%.
We note that the emergence of human-level abilities in these domains has recently been observed with the
latest generation of LLMs, e.g., see [LAD+22, SAT+22] for Google's PaLM on respectively mathematics
and medicine, and [BIK22] for GPT-3.5 on in law. Our approach to study GPT-4 is dierent from these
works, as we explained previously.
3. In Section 5, we test the model's ability to plan as well as to some extent to learn from experience by
ipping the table, simulate a game environment), as well as interact
with tools. In particular, the fact that GPT-4 can use tools (inclu

In [14]:
for doc in docs:
    print(doc.metadata)

{'source': 'data/SparkOfGPT-4.pdf', 'page': 8}
{'source': 'data/SparkOfGPT-4.pdf', 'page': 8}
{'source': 'data/SparkOfGPT-4.pdf', 'page': 70}
{'source': 'data/SparkOfGPT-4.pdf', 'page': 71}
{'source': 'data/SparkOfGPT-4.pdf', 'page': 71}


**Hybrid Search**

In [13]:
# Perform a hybrid search
docs = vector_store.similarity_search(
    query="What are the areas of improvement for GPT-4 over GPT-3.5?",
    k=5,
    search_type="hybrid",
)
print(docs[0].page_content)

Figure 1.5: GPT-4 passes mock technical interviews on LeetCode. GPT-4 could potentially be hired
as a software engineer3.
preliminary tests (see [Ope23] for much more) on the multiple choice component (majority of the score)
of the US Medical Licensing Exam Step 1, 2, and 3 with an accuracy around 80% in each. A similar
preliminary test of GPT-4's competency on the Multistate Bar Exam showed an accuracy above 70%.
We note that the emergence of human-level abilities in these domains has recently been observed with the
latest generation of LLMs, e.g., see [LAD+22, SAT+22] for Google's PaLM on respectively mathematics
and medicine, and [BIK22] for GPT-3.5 on in law. Our approach to study GPT-4 is dierent from these
works, as we explained previously.
3. In Section 5, we test the model's ability to plan as well as to some extent to learn from experience by
ipping the table, simulate a game environment), as well as interact
with tools. In particular, the fact that GPT-4 can use tools (inclu

In [42]:
len(docs)

5

In [43]:
docs[0]

Document(page_content='Figure 1.3: We queried GPT-4 three times, at roughly equal time intervals over the span of a month\nwhile the system was being re\x0cned, with the prompt \\Draw a unicorn in TikZ". We can see a clear\nevolution in the sophistication of GPT-4\'s drawings.\nis backed up by a rich theoretical framework [SSBD14, MRT18]. However, this methodology is not necessarily\nsuitable for studying GPT-4, for two reasons. First, since we do not have access to the full details of its vast\ntraining data, we have to assume that it has potentially seen every existing benchmark, or at least some similar\ndata. For example, it seems like GPT-4 knows the recently proposed BIG-bench [SRR+22] (at least GPT-4\nknows the canary GUID from BIG-bench). Of course, OpenAI themselves have access to all the training\ndetails, and thus their report [Ope23] contains a lot of detailed benchmark results. Nevertheless, the second\nreason for going beyond traditional benchmarks is probably more signi\

**Use llm to answers the query**

In [44]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
# from langchain.document_loaders import CSVLoader
# from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [45]:
llm = ChatOpenAI(engine='gpt-4', temperature = 0.0)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [46]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [47]:
response = llm.call_as_llm(f"{qdocs} Question: What are the areas \
of improvement for GPT-4 over GPT-3.5?") 

In [48]:
display(Markdown(response))

GPT-4 demonstrates significant improvements over GPT-3.5 in various aspects, including:

1. Generality: GPT-4 shows a deeper and more flexible understanding of concepts, skills, and domains, enabling it to perform tasks that go beyond the typical scope of narrow AI systems.

2. Problem-solving: GPT-4 exhibits better problem-solving abilities, including mathematical reasoning, coding, and planning.

3. Human understanding: GPT-4 has a better grasp of human behavior, emotions, and communication, making it more effective in understanding and interacting with humans.

4. Common sense: GPT-4 demonstrates improved common sense reasoning, which has been a challenge for previous language models.

5. Truthfulness: GPT-4 generates more accurate and truthful responses compared to GPT-3.5, making it more reliable for various tasks.

6. Performance on benchmarks: GPT-4 outperforms GPT-3.5 on several benchmarks, including those related to medicine, law, and software engineering.

However, it is essential to note that GPT-4 still has limitations, such as the lack of an "inner dialogue" or "scratchpad" for multi-step computations and the inability to store intermediate results. Additionally, the autoregressive architecture of GPT-4 may limit its planning capabilities.